# Introduction

The goal of this project is to visualise, how the impact of Players born NOT in USA has changed over the years. 

This project uses a kaggle datasets: https://www.kaggle.com/drgilermo/nba-players-stats. We can find 3 .csv files there. Seasons_stats.csv contains stats from every NBA player each season since 1949/1950. Two other files contain some basic informations about each player such as: name, birthdate, height, etc.
The list of international Players comes from Wikipedia: https://en.wikipedia.org/wiki/List_of_foreign_NBA_players

Important note: There is a lot of data that is missing, because not every stat is available since the first season. More details and explaination of each stat: https://www.basketball-reference.com/about/glossary.html

### Imports
Import libraries.

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

# Project specific imports
from unidecode import unidecode

# Analysis/Modeling
### Import statistics

In [2]:
stat = pd.read_csv('Seasons_Stats.csv')

### Basic info

In [3]:
stat.shape
stat.dtypes

(24691, 53)

Unnamed: 0      int64
Year          float64
Player         object
Pos            object
Age           float64
Tm             object
G             float64
GS            float64
MP            float64
PER           float64
TS%           float64
3PAr          float64
FTr           float64
ORB%          float64
DRB%          float64
               ...   
2PA           float64
2P%           float64
eFG%          float64
FT            float64
FTA           float64
FT%           float64
ORB           float64
DRB           float64
TRB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PTS           float64
Length: 53, dtype: object

### Cleaning

In [4]:
stat = stat.rename(columns={'Unnamed: 0': 'Rk'})
stat.isna().sum()

Rk           0
Year        67
Player      67
Pos         67
Age         75
Tm          67
G           67
GS        6458
MP         553
PER        590
TS%        153
3PAr      5852
FTr        166
ORB%      3899
DRB%      3899
          ... 
2PA         67
2P%        195
eFG%       166
FT          67
FTA         67
FT%        925
ORB       3894
DRB       3894
TRB        379
AST         67
STL       3894
BLK       3894
TOV       5046
PF          67
PTS         67
Length: 53, dtype: int64

Dropping rows where Player is NaN

TODO: Do I have to explain it?

In [5]:
stat = stat.dropna(subset=['Player'])

Dropping two columns with no values

<span style="color:red">Question: Do I have to explain, how do I know that or should I proove it?</span>

In [6]:
stat = stat.drop(['blanl', 'blank2'], axis=1)

# delete asteriks from Hall of Famers
stat['Player'] = stat['Player'].str.replace('*', '')

### Import stats from 2018

Source:


stats_2018: https://www.basketball-reference.com/leagues/NBA_2018_totals.html

adv_2018: https://www.basketball-reference.com/leagues/NBA_2018_advanced.html

In [7]:
stats_2018 = pd.read_csv('ignore/stats_2018.csv')
adv_2018 = pd.read_csv('ignore/adv_2018.csv')

Today I've spent a lot of time here trying to highlight all the non stat rows. I wanted to show, why I delete them.

In [48]:
stats_2018.loc[stats_2018.Player == 'Player'].style.applymap(lambda x: 'background-color: red')

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,Year


### Find rows without stats and delete them

In [9]:
stats_rows_to_delete = stats_2018.loc[stats_2018['Player'] == 'Player']
adv_rows_to_delete = adv_2018.loc[adv_2018['Player'] == 'Player']

stats_2018 = stats_2018.drop(stats_rows_to_delete.index)
adv_2018 = adv_2018.drop(adv_rows_to_delete.index)

Delete columns containing 'Unnamed'

In [10]:
adv_2018 = adv_2018.loc[:, ~adv_2018.columns.str.contains('Unnamed')]

In [11]:
cols = adv_2018.columns.difference(stats_2018.columns)
cols

Index(['3PAr', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM',
       'ORB%', 'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP',
       'WS', 'WS/48'],
      dtype='object')

In [12]:
stats_2018 = stats_2018.join(adv_2018[cols])
stats_2018.shape
stats_2018.dtypes

(664, 50)

Rk        object
Player    object
Pos       object
Age       object
Tm        object
G         object
GS        object
MP        object
FG        object
FGA       object
FG%       object
3P        object
3PA       object
3P%       object
2P        object
           ...  
DRB%      object
DWS       object
FTr       object
OBPM      object
ORB%      object
OWS       object
PER       object
STL%      object
TOV%      object
TRB%      object
TS%       object
USG%      object
VORP      object
WS        object
WS/48     object
Length: 50, dtype: object

In [13]:
cols = stats_2018.columns.values[5:]
stats_2018[cols] = stats_2018[cols].apply(pd.to_numeric, errors='coerce')
stats_2018.dtypes

Rk         object
Player     object
Pos        object
Age        object
Tm         object
G           int64
GS          int64
MP          int64
FG          int64
FGA         int64
FG%       float64
3P          int64
3PA         int64
3P%       float64
2P          int64
           ...   
DRB%      float64
DWS       float64
FTr       float64
OBPM      float64
ORB%      float64
OWS       float64
PER       float64
STL%      float64
TOV%      float64
TRB%      float64
TS%       float64
USG%      float64
VORP      float64
WS        float64
WS/48     float64
Length: 50, dtype: object

#### A very handy function to find differences in column names

In [14]:
stat.columns.difference(stats_2018.columns)

Index(['Year'], dtype='object')

In [15]:
stats_2018['Year'] = 2018
stats_2018.tail()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,...,BLK,TOV,PF,PTS,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,Year
685,537,Tyler Zeller,C,28,BRK,42,33,703,125,229,0.546,10,26,0.385,115,203,0.567,0.568,40,60,0.667,63,131,194,28,...,21,35,78,300,0.114,6.5,2.2,-2.5,-0.6,20.0,0.6,0.262,-1.9,9.4,1.0,15.3,0.6,12.1,14.7,0.587,17.9,-0.1,1.5,0.105,2018
686,537,Tyler Zeller,C,28,MIL,24,1,406,62,105,0.590,0,2,0.000,62,103,0.602,0.590,17,19,0.895,47,64,111,19,...,14,12,48,141,0.019,7.0,2.9,-0.1,-0.4,18.4,0.3,0.181,0.3,13.6,1.1,17.1,0.9,9.6,16.0,0.622,13.9,0.2,1.4,0.163,2018
687,538,Paul Zipser,SF,23,CHI,54,12,824,81,234,0.346,37,110,0.336,44,124,0.355,0.425,19,25,0.760,13,118,131,46,...,15,43,86,218,0.470,8.0,1.6,-5.9,-0.3,16.0,0.6,0.107,-5.5,1.6,-1.1,5.2,1.2,14.9,8.5,0.445,15.2,-0.8,-0.6,-0.034,2018
688,539,Ante Žižić,C,21,CLE,32,2,214,49,67,0.731,0,0,NaN,49,67,0.731,0.731,21,29,0.724,24,36,60,5,...,13,11,30,119,0.000,3.8,5.2,0.1,-1.2,18.6,0.2,0.433,1.3,12.8,0.9,24.2,0.5,12.1,15.7,0.746,18.8,0.1,1.0,0.231,2018
689,540,Ivica Zubac,C,20,LAL,43,0,410,61,122,0.500,0,1,0.000,61,121,0.504,0.500,39,51,0.765,45,78,123,25,...,15,26,47,161,0.008,8.8,3.0,-2.2,0.5,20.1,0.5,0.418,-2.7,11.8,0.5,15.3,0.9,15.3,16.0,0.557,17.6,0.0,1.0,0.118,2018


### Import stats from 2019

Source:

stats_2019: https://www.basketball-reference.com/leagues/NBA_2019_totals.html

adv_2019: https://www.basketball-reference.com/leagues/NBA_2019_advanced.html

In [16]:
stats_2019 = pd.read_csv('ignore/stats_2019.csv')
adv_2019 = pd.read_csv('ignore/adv_2019.csv')

### Find rows without stats and delete them

In [17]:
stats_rows_to_delete = stats_2019.loc[stats_2019['Player'] == 'Player']
adv_rows_to_delete = adv_2019.loc[adv_2019['Player'] == 'Player']

stats_2019 = stats_2019.drop(stats_rows_to_delete.index)
adv_2019 = adv_2019.drop(adv_rows_to_delete.index)

Delete columns containing 'Unnamed'

In [18]:
adv_2019 = adv_2019.loc[:, ~adv_2019.columns.str.contains('Unnamed')]

In [19]:
cols = adv_2019.columns.difference(stats_2019.columns)
cols

Index(['3PAr', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM',
       'ORB%', 'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP',
       'WS', 'WS/48'],
      dtype='object')

In [20]:
stats_2019 = stats_2019.join(adv_2019[cols])
stats_2019.shape

(708, 50)

In [21]:
cols = stats_2019.columns.values[5:]
stats_2019[cols] = stats_2019[cols].apply(pd.to_numeric, errors='coerce')
stats_2019.dtypes

Rk         object
Player     object
Pos        object
Age        object
Tm         object
G           int64
GS          int64
MP          int64
FG          int64
FGA         int64
FG%       float64
3P          int64
3PA         int64
3P%       float64
2P          int64
           ...   
DRB%      float64
DWS       float64
FTr       float64
OBPM      float64
ORB%      float64
OWS       float64
PER       float64
STL%      float64
TOV%      float64
TRB%      float64
TS%       float64
USG%      float64
VORP      float64
WS        float64
WS/48     float64
Length: 50, dtype: object

#### A very handy function to find differences in column names

In [22]:
stat.columns.difference(stats_2019.columns)

Index(['Year'], dtype='object')

In [23]:
stats_2019['Year'] = 2019
stats_2019.sample(5)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,...,BLK,TOV,PF,PTS,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,Year
288,213,Andrew Harrison,PG,24,CLE,10,0,144,12,39,0.308,3,14,0.214,9,25,0.360,0.346,16,16,1.000,4,11,15,17,...,2,10,26,43,0.359,16.3,1.2,-6.2,-2.2,8.9,0.0,0.410,-4.0,3.0,-0.1,6.6,1.4,17.8,5.8,0.467,17.0,-0.2,-0.1,-0.022,2019
627,453,Dennis Smith,PG,21,TOT,53,50,1508,278,650,0.428,67,208,0.322,211,442,0.477,0.479,99,156,0.635,32,123,155,252,...,20,154,129,722,0.320,27.3,1.1,-2.5,-0.9,8.7,1.2,0.240,-1.7,2.2,-1.2,12.3,2.1,17.6,5.5,0.502,24.8,-0.2,0.0,0.001,2019
393,288,Kyle Korver,PF,37,CLE,16,0,251,35,76,0.461,25,54,0.463,10,22,0.455,0.625,13,16,0.813,0,29,29,17,...,2,17,22,108,0.711,10.1,0.7,-3.7,-3.4,13.4,0.0,0.211,-0.3,0.0,0.2,11.3,0.6,17.0,6.4,0.650,17.4,-0.1,0.2,0.043,2019
557,403,Theo Pinson,SG,23,BRK,18,0,211,25,73,0.342,12,46,0.261,13,27,0.481,0.425,19,22,0.864,4,32,36,21,...,0,18,15,81,0.630,14.1,0.0,-4.8,-1.0,15.9,0.2,0.301,-3.7,2.0,-0.2,8.1,1.4,17.9,9.0,0.490,20.0,-0.1,0.0,0.004,2019
477,347,C.J. Miles,SF,31,MEM,13,0,294,42,105,0.400,24,66,0.364,18,39,0.462,0.514,13,14,0.929,1,26,27,14,...,5,9,17,121,0.629,7.4,1.6,-2.9,-1.5,10.1,0.3,0.133,-1.4,0.4,0.1,10.8,1.4,7.5,5.2,0.544,18.3,-0.1,0.3,0.054,2019


In [24]:
stat.columns.difference(stats_2019.columns)

Index([], dtype='object')

### Add stats from last two years

In [25]:
stat = pd.concat([stat, stats_2018], sort=False)
stat = stat.reset_index(drop=True)

stat = pd.concat([stat, stats_2019], sort=False)
stat = stat.reset_index(drop=True)

In [26]:
stat.shape
stat.sample(5)

(25996, 51)

,Rk,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,...,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
665,668,1953.0,Don Henriksen,C,23,BLB,68.0,NaN,2263.0,12.9,0.479,NaN,0.592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.9,0.5,3.5,0.074,...,NaN,NaN,NaN,199.0,475.0,0.419,NaN,NaN,NaN,199.0,475.0,0.419,0.419,176.0,281.0,0.626,NaN,NaN,506.0,129.0,NaN,NaN,NaN,242.0,574.0
23791,23857,2016.0,Kendall Marshall,PG,24,PHI,30.0,6.0,400.0,6.7,0.449,0.415,0.110,0.5,7.3,3.8,28.4,2.0,0.4,25.8,18.3,-0.7,0.1,-0.5,-0.064,...,-3.3,-8.8,-0.7,43.0,118.0,0.364,16.0,49.0,0.327,27.0,69.0,0.391,0.432,9.0,13.0,0.692,2.0,26.0,28.0,73.0,16.0,2.0,43.0,30.0,111.0
4745,4772,1977.0,Leonard Gray,PF,25,WSB,58.0,NaN,996.0,10.2,0.475,NaN,0.242,6.4,12.6,9.6,9.3,1.4,0.9,NaN,NaN,-0.2,1.1,0.9,0.042,...,0.2,-2.3,-0.1,144.0,330.0,0.436,NaN,NaN,NaN,144.0,330.0,0.436,0.436,59.0,80.0,0.738,61.0,125.0,186.0,69.0,31.0,18.0,NaN,189.0,347.0
5305,5333,1978.0,David Thompson,SG,23,DEN,80.0,NaN,3025.0,23.2,0.578,NaN,0.422,5.5,7.7,6.6,19.3,1.3,1.7,11.5,26.8,10.8,1.9,12.7,0.202,...,-1.5,3.6,4.3,826.0,1584.0,0.521,NaN,NaN,NaN,826.0,1584.0,0.521,0.521,520.0,668.0,0.778,156.0,234.0,390.0,362.0,92.0,99.0,245.0,213.0,2172.0
19469,19528,2009.0,Francisco Garcia,SF,27,SAC,65.0,36.0,1975.0,14.1,0.554,0.332,0.268,3.3,9.6,6.4,12.6,2.1,2.5,13.1,19.4,1.7,0.7,2.4,0.058,...,-0.4,-0.1,0.9,295.0,665.0,0.444,88.0,221.0,0.398,207.0,444.0,0.466,0.510,146.0,178.0,0.820,56.0,162.0,218.0,150.0,80.0,67.0,112.0,196.0,824.0


### Replace International letters

In [27]:
stat['Player'] = stat['Player'].apply(lambda x: unidecode(x))

## Import the list of all time international players

Source: https://en.wikipedia.org/wiki/List_of_foreign_NBA_players

In [28]:
intp = pd.read_csv('international.csv')

In [29]:
intp.shape
intp.dtypes

(698, 8)

Nationality[A]      object
Birthplace[B]       object
Player              object
Pos.                object
Career[C]           object
Yrs                float64
Notes               object
Ref.                object
dtype: object

In [30]:
intp.sample(10)

,Nationality[A],Birthplace[B],Player,Pos.,Career[C],Yrs,Notes,Ref.
335,Israel,—,T. J. Leaf*,F,2017–present,2.0,"Born in Israel to American parents, previously...",[349]
2,Argentina,—,Carlos Delfino,G,2004–2008;,8.0,—,[13]
360,NaN,NaN,(formerly J. R. Henderson),NaN,NaN,NaN,NaN,NaN
222,France,—,Evan Fournier*,G/F,2012–present,7.0,—,[230]
177,Cuba,—,Lazaro Borrell,F,1999–2000,1.0,—,[191]
134,China,—,Mengke Bateer,C,2002–2003,3.0,—,[161]
544,NaN,(now Serbia),NaN,NaN,NaN,NaN,NaN,NaN
447,NaN,(now North Macedonia),NaN,NaN,NaN,NaN,NaN,NaN
417,NaN,(now Montenegro),NaN,NaN,NaN,NaN,NaN,NaN
157,NaN,(now Bosnia and Herzegovina),NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning

Rename columns. Drop 2 of them and all Player NaNs

In [31]:
intp.columns = ['Nationality', 'Birthplace', 'Player', 'Pos', 'Career', 'Years', 'Notes', 'Ref']
intp = intp.drop(['Notes', 'Ref'], axis=1)
intp = intp.dropna(subset=['Player', 'Nationality'])

Drop Players with US Nationality or born in USA


In [32]:
intp = intp.loc[intp.Birthplace != ' United States ']
intp = intp.loc[intp.Nationality != ' United States ']

In [33]:
intp.shape
intp.index = pd.RangeIndex(len(intp))
intp.sample(10)

(420, 6)

,Nationality,Birthplace,Player,Pos,Career,Years
335,Serbia,SFR Yugoslavia,Predrag Danilović,G,1995–1997,2.0
56,Brazil,—,Nenê *,F/C,2002–present,17.0
3,Argentina,—,Manu Ginóbili,G,2002–2018,16.0
402,Turkey,—,Mehmet Okur,F/C,2002–2012,10.0
291,Nigeria,—,Michael Olowokandi,C,1998–2007,9.0
109,China,—,Yao Ming^,C,2002–2011,8.0
34,Australia,—,Ben Simmons*,F,2017–present,2.0
132,Cuba,—,Lazaro Borrell,F,1999–2000,1.0
127,Croatia,SFR Yugoslavia,Žan Tabak,C,1994–1998; 1999–2001,6.0
208,Great Britain,—,James Donaldson,C,1980–1993; 1995,14.0


### Check which Names match those from stats data frame

In [34]:
intp.loc[intp.Player.isin(stat.Player)]

,Nationality,Birthplace,Player,Pos,Career,Years
156,France,—,Tariq Abdul-Wahad,F,1997–2003,6.0


Only one player matches. Apparently on the list from Wikipedia there is a trailing space and some other additional signs. Let's get rid of them

In [35]:
intp['Player'] = intp['Player'].str.replace('[*^]', '')
# intp['Player'] = intp['Player'].str.replace('^', '')
intp['Player'] = np.where(intp.Player.str.endswith(' '), intp.Player.str[:-1], intp.Player)

### Decode international letters

In [36]:
intp['Player'] = intp['Player'].apply(lambda x: unidecode(x))

In [37]:
intp.Player.isin(stat.Player).value_counts()


True     395
False     25
Name: Player, dtype: int64

Checking, which players still don't match to those from stats and played more than two seasons

In [38]:
# For myself: Never forget parenthesses when using multiple conditions
intp.loc[(~intp.Player.isin(stat.Player)) & (intp.Years > 2.0)]

,Nationality,Birthplace,Player,Pos,Career,Years
41,Belgium,Zaire,Didier Mbenga,C,2004–2011,7.0
56,Brazil,—,Nene,F/C,2002–present,17.0
123,Croatia,SFR Yugoslavia,Dino Rada,F/C,1993–1997,4.0
201,Germany,West Germany,Christian Welp,C,1987–1990,3.0
301,Puerto Rico,—,Jose Juan Barea,G,2006–present,13.0
313,Russia,Soviet Union,Victor Khryapa,F,2004–2008,4.0
364,Slovenia,SFR Yugoslavia,Radoslav Nesterovic,C,1998–2010,12.0
409,Ukraine,Soviet Union,Slava Medvedenko,F,2000–2007,7.0


Adding those manually

<span style="color:red">Question: Can I do something like this or is it just bad practice? </span>

In [39]:
intp.loc[intp.Player == 'Nene', 'Player'] = 'Nene Hilario'
intp.loc[intp.Player == 'Radoslav Nesterovic', 'Player'] = 'Rasho Nesterovic'
intp.loc[intp.Player == 'Jose Juan Barea', 'Player'] = 'J.J. Barea'
intp.loc[intp.Player == 'Didier Mbenga', 'Player'] = 'Didier Ilunga-Mbenga'
intp.loc[intp.Player == 'Slava Medvedenko', 'Player'] = 'Stanislav Medvedenko'
intp.loc[intp.Player == 'Jakob Poltl', 'Player'] = 'Jakob Poeltl'
intp.loc[intp.Player == 'Victor Khryapa', 'Player'] = 'Viktor Khryapa'
intp.loc[intp.Player == 'Dino Rada', 'Player'] = 'Dino Radja'
intp.loc[intp.Player == 'Luc Mbah a Moute', 'Player'] = 'Luc Mbah'
intp.loc[intp.Player == 'Christian Welp', 'Player'] = 'Chris Welp'

intp.Player.isin(stat.Player).value_counts()
# intp.loc[~intp.Player.isin(stat.Player)].sample(20)

True     403
False     17
Name: Player, dtype: int64

In [40]:
stati = stat.loc[stat.Player.isin(intp.Player)]
stati.shape
stat_by_year = stat.groupby(stat.Year).sum()
stati_by_year = stati.groupby(stati.Year).sum()

(2287, 51)

In [41]:
stati_by_year['PTS'].iplot(kind='line', xTitle='Year', yTitle='Points total')

In [42]:
trace1 = go.Scatter(
    x=stat_by_year.index,
    y=stat_by_year.PTS,
    name='Totals'
)
trace2 = go.Scatter(
    x=stati_by_year.index,
    y=stati_by_year.PTS,
    name='Non-US'
)

data = [trace1, trace2]
layout = go.Layout(
    title=go.layout.Title(
        text='Total points per year',
        xref='paper',
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Year',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#5f5f5f'
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Total Points',
            font=dict(
                family='Courier New, monospace',
                size=18,
                color='#5f5f5f'
            )
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

In [43]:
stati_by_year

,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1950.0,42.0,0.0,0.0,0.0,0.466,0.000,0.359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.9,3.9,0.000,0.0,0.0,0.0,0.0,164.0,390.0,0.421,0.0,0.0,0.000,164.0,390.0,0.421,0.421,93.0,140.0,0.664,0.0,0.0,0.0,78.0,0.0,0.0,0.0,126.0,421.0
1951.0,44.0,0.0,0.0,0.0,0.446,0.000,0.289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.6,2.1,0.000,0.0,0.0,0.0,0.0,135.0,336.0,0.402,0.0,0.0,0.000,135.0,336.0,0.402,0.402,68.0,97.0,0.701,0.0,0.0,195.0,121.0,0.0,0.0,0.0,144.0,338.0
1952.0,57.0,0.0,1507.0,16.6,0.497,0.000,0.350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7,1.2,4.8,0.154,0.0,0.0,0.0,0.0,200.0,457.0,0.438,0.0,0.0,0.000,200.0,457.0,0.438,0.438,124.0,160.0,0.775,0.0,0.0,264.0,164.0,0.0,0.0,0.0,188.0,524.0
1953.0,61.0,0.0,1745.0,18.8,0.499,0.000,0.390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.7,2.3,7.1,0.194,0.0,0.0,0.0,0.0,272.0,625.0,0.435,0.0,0.0,0.000,272.0,625.0,0.435,0.435,187.0,244.0,0.766,0.0,0.0,342.0,144.0,0.0,0.0,0.0,242.0,731.0
1954.0,155.0,0.0,4211.0,55.1,1.740,0.000,1.786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.2,1.5,5.8,0.256,0.0,0.0,0.0,0.0,455.0,1227.0,1.417,0.0,0.0,0.000,455.0,1227.0,1.417,1.417,405.0,563.0,2.992,0.0,0.0,770.0,275.0,0.0,0.0,0.0,456.0,1315.0
1955.0,64.0,0.0,1326.0,0.0,0.456,0.000,0.472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,148.0,386.0,0.383,0.0,0.0,0.000,148.0,386.0,0.383,0.383,129.0,182.0,0.709,0.0,0.0,297.0,86.0,0.0,0.0,0.0,180.0,425.0
1956.0,157.0,0.0,2452.0,40.5,1.766,0.000,1.636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.2,2.9,5.2,-0.126,0.0,0.0,0.0,0.0,331.0,904.0,1.498,0.0,0.0,0.000,331.0,904.0,1.498,1.498,339.0,475.0,3.543,0.0,0.0,515.0,259.0,0.0,0.0,0.0,248.0,1001.0
1957.0,60.0,0.0,1592.0,17.6,0.489,0.000,0.400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3,1.2,4.5,0.134,0.0,0.0,0.0,0.0,253.0,585.0,0.432,0.0,0.0,0.000,253.0,585.0,0.432,0.432,167.0,234.0,0.714,0.0,0.0,401.0,113.0,0.0,0.0,0.0,118.0,673.0
1958.0,17.0,0.0,302.0,11.5,0.410,0.000,0.314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.031,0.0,0.0,0.0,0.0,49.0,137.0,0.358,0.0,0.0,0.000,49.0,137.0,0.358,0.358,30.0,43.0,0.698,0.0,0.0,65.0,19.0,0.0,0.0,0.0,36.0,128.0


In [44]:
grouped = stati.groupby(['Year', 'Player']).count()
traded = grouped.loc[grouped.G > 1]
traded

Rk  Pos  Age  Tm  G  GS  MP  PER  TS%  3PAr  \
Year   Player                                                                 
1954.0 Bob Houbregs              3    3    3   3  3   0   3    3    3     0   
1956.0 Chris Harris              3    3    3   3  3   0   3    3    3     0   
1979.0 Butch Lee                 3    3    3   3  3   0   3    3    3     0   
1980.0 Butch Lee                 3    3    3   3  3   0   3    3    3     3   
1986.0 James Donaldson           3    3    3   3  3   3   3    3    3     3   
       Ron Crevier               3    3    3   3  3   3   3    3    3     3   
1987.0 Mychal Thompson           3    3    3   3  3   3   3    3    3     3   
1989.0 Detlef Schrempf           3    3    3   3  3   3   3    3    3     3   
1990.0 Chris Welp                3    3    3   3  3   3   3    3    3     3   
       Uwe Blab                  3    3    3   3  3   3   3    3    3     3   
1991.0 Drazen Petrovic           3    3    3   3  3   3   3    3    3     3   
       Mike Smrek                3    3    3   3  3   3   3    3    3     3   
       Olden Polynice            3    3    3   3  3   3   3    3    3     3   
1992.0 James Donaldson           3    3    3   3  3   3   3    3    3     3   
1993.0 Alaa Abdelnaby            3    3    3   3  3   3   3    3    3     3   
...                             ..  ...  ...  .. ..  ..  ..  ...  ...   ...   
2019.0 Boban Marjanovic          3    3    3   3  3   3   3    3    3     3   
       Dario Saric               3    3    3   3  3   3   3    3    3     3   
       Enes Kanter               3    3    3   3  3   3   3    3    3     3   
       Ivica Zubac               3    3    3   3  3   3   3    3    3     3   
       Jonas Valanciunas         3    3    3   3  3   3   3    3    3     3   
       Marc Gasol                3    3    3   3  3   3   3    3    3     3   
       Matthew Dellavedova       3    3    3   3  3   3   3    3    3     3   
       Mitch Creek               3    3    3   3  3   3   3    3    3     3   
       Nik Stauskas              3    3    3   3  3   3   3    3    3     3   
       Nikola Mirotic            3    3    3   3  3   3   3    3    3     3   
       Pau Gasol                 3    3    3   3  3   3   3    3    3     3   
       Skal Labissiere           3    3    3   3  3   3   3    3    3     3   
       Sviatoslav Mykhailiuk     3    3    3   3  3   3   3    3    3     3   
       Thon Maker                3    3    3   3  3   3   3    3    3     3   
       Timothe Luwawu-Cabarrot   3    3    3   3  3   3   3    3    3     3   

                                FTr  ORB%  DRB%  TRB%  AST%  STL%  BLK%  TOV%  \
Year   Player                                                                   
1954.0 Bob Houbregs               3     0     0     0     0     0     0     0   
1956.0 Chris Harris               3     0     0     0     0     0     0     0   
1979.0 Butch Lee                  3     3     3     3     3     3     3     3   
1980.0 Butch Lee                  3     3     3     3     3     3     3     3   
1986.0 James Donaldson            3     3     3     3     3     3     3     3   
       Ron Crevier                3     3     3     3     3     3     3     3   
1987.0 Mychal Thompson            3     3     3     3     3     3     3     3   
1989.0 Detlef Schrempf            3     3     3     3     3     3     3     3   
1990.0 Chris Welp                 3     3     3     3     3     3     3     3   
       Uwe Blab                   3     3     3     3     3     3     3     3   
1991.0 Drazen Petrovic            3     3     3     3     3     3     3     3   
       Mike Smrek                 3     3     3     3     3     3     3     3   
       Olden Polynice             3     3     3     3     3     3     3     3   
1992.0 James Donaldson            3     3     3     3     3     3     3     3   
1993.0 Alaa Abdelnaby             3     3     3     3     3     3     3     3   
...                             ...   ...   ...

In [45]:
traded2 = stat.loc[stat.Tm == 'TOT']
# traded2
stati.isin(traded2)

,Rk,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,...,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
294,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
479,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
610,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
768,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
823,True,True,True,True,True,True,True,False,True,True,True,False,True,False,False,False,False,False,False,False,False,True,True,True,True,...,False,False,False,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,False,True,True
824,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
825,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
896,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
949,True,True,True,True,True,True,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,False,True,True
1051,True,True,True,True,True,True,True,False,True,True,True,False,True,False,False,False,False,False,False,False,False,True,True,True,True,...,False,False,False,True,True,True,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,False,True,True


# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here